# Building Agents with custom tools and Ollama 3.1

## What You'll Learn

In this tutorial, you will learn how to:
1. **Set Up the Environment**: Prepare your development environment with the necessary tools and libraries.
2. **Define Custom Tools**: Create custom functions that can be used by the agent to perform specific tasks.
3. **Integrate with Ollama Model**: Use the Ollama 3.1 model to generate responses based on user queries and tool descriptions.
4. **Build an Agent**: Combine the tools and the model into an intelligent agent that can process and respond to user queries.

## Basic Concepts

### 1. Intelligent Agents

An intelligent agent is a system that perceives its environment and takes actions to achieve specific goals. In this tutorial, our agent will process user inputs and use predefined tools to provide responses. This concept draws inspiration from various frameworks, including task-driven autonomous agents as explored in [this article by Yohei Nakajima](https://yoheinakajima.com/task-driven-autonomous-agent-utilizing-gpt-4-pinecone-and-langchain-for-diverse-applications/).

### 2. Custom Tools

Custom tools are user-defined functions that perform specific tasks, such as calculations or data manipulation. These tools will be utilized by the agent to process queries.

### 3. Language Models

Language models, like Ollama 3.1, are AI models trained to understand and generate human-like text. In this tutorial, the model will help the agent interpret user inputs and determine which tool to use.

### 3.1. Llama 3.1

**Llama 3.1**, released on July 23, 2024, is Meta's most advanced open-source language model to date. It boasts a massive 405 billion parameters and is designed for a wide range of applications, including general knowledge, steerability, and multilingual translation. With enhanced context length support up to 128K and state-of-the-art capabilities, Llama 3.1 aims to democratize AI by providing developers with the tools to create custom agents and innovative applications. More details can be found in the [official announcement](https://ai.meta.com/blog/meta-llama-3-1/).

## 1. Setting Up the Environment

Before we dive into building our agent, we need to set up the necessary environment. This involves installing required packages and ensuring our Python environment is ready for development.

In [21]:
# Install termcolor for colored terminal outputs
%pip install termcolor

Note: you may need to restart the kernel to use updated packages.


In [22]:
# Import necessary libraries
from termcolor import colored
import operator
import json
import requests
import os

## 2. Defining the System Prompt and Model Configuration

In this section, we define the system prompt and set up the configuration for the Ollama model. The system prompt provides context and instructions for the model, ensuring it understands how to use the available tools and respond to queries.

### System Prompt Template

The system prompt is a crucial component in guiding the behavior of the language model. It provides a structured way to present the available tools and define how the model should respond to user queries.

*Based on:*
[Model Cards & Prompt formats Llama 3.1](https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/#user-defined-custom-tool-calling)

In [23]:
agent_system_prompt_template = """
Environment: ipython\n

Cutting Knowledge Date: December 2023
Today Date: July 2024

You have access to the following functions:

{tool_descriptions}
Think very carefully before calling functions.
If you choose to call a function ONLY reply in the JSON format with no prefix or suffix:

"tool_choice": "name_of_the_tool",
"tool_input": "inputs_to_the_tool"

- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox 
                or "no tool" if you do not need to use a tool.
- `tool_input`: The specific inputs required for the selected tool. 
                If no tool, just provide a response to the query.

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

You are a helpful Assistant.
"""

### Setting Up the Ollama Model

The `setup_ollama_model` function configures the model settings, including the endpoint, model name, system prompt, and other parameters. This setup is essential for initializing the model with the correct configuration, ensuring it can process queries and utilize the tools effectively.

In [24]:
def setup_ollama_model(model, system_prompt, temperature=0, stop=None):
    """
    Sets up the Ollama model configuration.

    Parameters:
    model (str): The name of the model to use.
    system_prompt (str): The system prompt to use.
    temperature (float): The temperature setting for the model.
    stop (str): The stop token for the model.

    Returns:
    dict: Configuration for the Ollama model.
    """
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "system_prompt": system_prompt,
        "temperature": temperature,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration
ollama_config = setup_ollama_model(
    model="llama3.1", system_prompt=agent_system_prompt_template
)

## 3. Creating custom tools

This section introduces custom tools that our agent will use to perform specific tasks. These tools are functions designed to handle particular operations, such as mathematical calculations and string manipulation.

### Basic Calculator

The `basic_calculator` function is designed to perform arithmetic operations on two numbers based on the input provided. It accepts a JSON string containing the numbers and the operation type.

In [25]:
def basic_calculator(input_str):
    """
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    input_str (str): A JSON string representing a dictionary with keys:
        - 'num1' (int): The first number.
        - 'num2' (int): The second number.
        - 'operation' (str): The operation to perform. Supported operations are 'add', 'subtract',
                             'multiply', 'divide', 'floor_divide', 'modulus', 'power', 'lt',
                             'le', 'eq', 'ne', 'ge', 'gt'.

    Returns:
    str: The formatted result of the operation.

    Raises:
    Exception: If an error occurs during the operation (e.g., division by zero).
    ValueError: If an unsupported operation is requested or input is invalid.
    """
    # Clean and parse the input string
    try:
        # Replace single quotes with double quotes
        input_str_clean = input_str.replace("'", '"')
        # Remove any extraneous characters such as trailing quotes
        input_str_clean = input_str_clean.strip().strip('"')

        input_dict = json.loads(input_str_clean)
        num1 = input_dict["num1"]
        num2 = input_dict["num2"]
        operation = input_dict["operation"]
    except (json.JSONDecodeError, KeyError) as e:
        return str(e), "Invalid input format. Please provide a valid JSON string."

    # Define the supported operations
    operations = {
        "add": operator.add,
        "subtract": operator.sub,
        "multiply": operator.mul,
        "divide": operator.truediv,
        "floor_divide": operator.floordiv,
        "modulus": operator.mod,
        "power": operator.pow,
        "lt": operator.lt,
        "le": operator.le,
        "eq": operator.eq,
        "ne": operator.ne,
        "ge": operator.ge,
        "gt": operator.gt,
    }

    # Check if the operation is supported
    if operation in operations:
        try:
            # Perform the operation
            result = operations[operation](num1, num2)
            result_formatted = (
                f"\n\nThe answer is: {result}.\nCalculated with basic_calculator."
            )
            return result_formatted
        except Exception as e:
            return str(e), "\n\nError during operation execution."
    else:
        return "\n\nUnsupported operation. Please provide a valid operation."

### Reverse String

The `reverse_string` function takes a string as input and returns its reverse. This simple tool is useful for tasks involving string manipulation.

In [26]:
def reverse_string(input_string):
    """
    Reverse the given string.

    Parameters:
    - input_string (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    # Reverse the string using slicing
    reversed_string = input_string[::-1]

    reversed_string = f"The reversed string is: {reversed_string}\n\nExecuted using the reverse_string function."
    # print (f"DEBUG: reversed_string: {reversed_string}")
    return reversed_string

In [27]:
# To use these tools within our agent, we register them in a list. This list will be referenced by the agent to determine which tools are available for use.
tools = [basic_calculator, reverse_string]

## 4. Building the Agent Class

This section focuses on constructing the `Agent` class, which integrates the tools and the Ollama model. The class handles user queries, determines the appropriate tools to use, and executes the necessary actions. This structure encapsulates the core functionalities of the agent, making it a reusable and maintainable component.

### Agent Class Definition

The `Agent` class is designed to manage the interaction between user queries, the available tools, and the language model. It includes methods for initializing the agent, preparing tool descriptions, generating responses, and executing tools.

- The agent is initialized with a list of tools and a configuration dictionary for the model.
- This includes setting up the model's endpoint, name, system prompt, temperature, headers, and stop token.
- The `prepare_tools` method is called to set up the descriptions of the tools.

**Preparing Tools**

The `prepare_tools` method processes the tools' docstrings to extract and format descriptions and parameter information. This is crucial for generating a system prompt that informs the model of the available tools.

**Generating Model Responses**

The `think` method generates responses by sending user queries and the system prompt to the Ollama model.

**Executing Tools**

The `execute_tool` method determines which tool to use based on the model's response and executes it.

**Main Method: Work**

The `work` method is the primary entry point for processing user queries. It integrates the thinking and execution steps.

In [28]:
class Agent:
    def __init__(self, tools, model_config):
        """
        Initializes the agent with a list of tools and a model configuration.

        Parameters:
        tools (list): List of tool functions.
        model_config (dict): Configuration for the model, including endpoint, model name, system prompt, etc.
        """
        self.tools = tools
        self.model_endpoint = model_config["model_endpoint"]
        self.model_name = model_config["model"]
        self.system_prompt = model_config["system_prompt"]
        self.temperature = model_config["temperature"]
        self.headers = model_config["headers"]
        self.stop = model_config["stop"]
        self.tool_descriptions = self.prepare_tools()

    def prepare_tools(self):
        """
        Prepares descriptions of the available tools.

        Returns:
        str: JSON-like formatted string describing the tools.
        """
        tools_info = []

        for tool in self.tools:
            name = tool.__name__
            docstring = (
                tool.__doc__.strip() if tool.__doc__ else "No description available."
            )
            parts = docstring.split("\n\n", 1)
            description = parts[0].split("\n")[0]
            parameters_section = parts[1] if len(parts) > 1 else ""

            param_dict = {}

            if "Parameters:" in parameters_section:
                parameters_lines = parameters_section.split("\n")
                param_section_started = False

                for line in parameters_lines:
                    if "Parameters:" in line:
                        param_section_started = True
                        continue
                    if not param_section_started or not line.strip():
                        continue

                    if ": " in line:
                        param_name_type_desc = line.split(":", 1)
                        if len(param_name_type_desc) == 2:
                            param_name_type, description = param_name_type_desc
                            param_name = param_name_type.split("(", 1)[0].strip()
                            param_type = (
                                param_name_type.split("(", 1)[1].split(")", 1)[0]
                                if "(" in param_name_type
                                else "str"
                            )
                            param_dict[param_name] = {
                                "param_type": param_type.strip(),
                                "description": description.strip(),
                                "required": True,
                            }

            tool_info = {
                "name": name,
                "description": description,
                "parameters": param_dict,
            }
            tools_info.append(tool_info)

        tools_description = "\n".join(
            [
                f"Use the function '{info['name']}' to: {info['description']}\n{json.dumps(info, indent=2)}"
                for info in tools_info
            ]
        )

        return tools_description

    def think(self, prompt):
        """
        Generates a response from the model based on the provided prompt.

        Parameters:
        prompt (str): The user query.

        Returns:
        dict: The response from the model.
        """
        payload = {
            "model": self.model_name,
            "format": "json",
            "prompt": prompt,
            "system": self.system_prompt.format(
                tool_descriptions=self.tool_descriptions
            ),
            "stream": False,
            "temperature": self.temperature,
            "stop": self.stop,
        }

        try:
            response = requests.post(
                self.model_endpoint, headers=self.headers, data=json.dumps(payload)
            )
            response_json = response.json()
            response_dict = json.loads(response_json["response"])

            print(f"\n\nResponse from model: {response_dict}")
            return response_dict
        except requests.RequestException as e:
            print(f"Error in invoking model! {str(e)}")
            return {"error": str(e)}

    def execute_tool(self, response):
        """
        Executes the appropriate tool based on the model's response.

        Parameters:
        response (dict): The model's response containing tool_choice and tool_input.

        Returns:
        None
        """
        tool_choice = response.get("tool_choice")
        tool_input = response.get("tool_input")

        for tool in self.tools:
            if tool.__name__ == tool_choice:
                try:
                    result = tool(tool_input)
                    print(colored(result, "cyan"))
                except Exception as e:
                    print(
                        colored(f"Error executing tool {tool_choice}: {str(e)}", "red")
                    )
                return

        print(colored(f"Tool {tool_choice} not found or unsupported operation.", "red"))

    def work(self, prompt):
        """
        The main method to process a user query, generate a response, and execute the appropriate tool.

        Parameters:
        prompt (str): The user query.

        Returns:
        None
        """
        response = self.think(prompt)
        self.execute_tool(response)

In [29]:
# Initialize the agent
agent = Agent(tools=tools, model_config=ollama_config)

## Running the Agent

This section demonstrates how to run the agent with example prompts. We will use the `work` method of the `Agent` class to process different types of user queries, showcasing the agent's ability to select and execute the appropriate tool.

### Example 1: Basic Calculation

In this example, we provide the agent with a mathematical query. The agent will determine the appropriate tool to use (in this case, `basic_calculator`) and execute it to provide the answer.

In [30]:
prompt = "What is 5+5?"
agent.work(prompt)



Response from model: {'tool_choice': 'basic_calculator', 'tool_input': '{"num1": 5, "operation": "add", "num2": 5}'}


The answer is: 10.
Calculated with basic_calculator.


### Example 2: String Manipulation

This example involves a string manipulation task where the user asks the agent to reverse a given string. The agent identifies and uses the `reverse_string` tool to accomplish this.

In [31]:
prompt = "Reverse this string: I want to be reversed!"
agent.work(prompt)



Response from model: {'tool_choice': 'reverse_string', 'tool_input': 'I want to be reversed!'}
The reversed string is: !desrever eb ot tnaw I

Executed using the reverse_string function.
